In [16]:
import pandas as pd
import numpy as np
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

In [17]:
df = pd.read_csv('../Datasets/cases/Human_Resources_Analytics/HR_comma_sep.csv')
tst = pd.read_csv('../Datasets/cases/Human_Resources_Analytics/tst_hr.csv')

In [18]:
X = df.drop('left', axis=1)
y = df['left']


obj.get_params() will give us the dictionary of parameters, useful for gridsearchcv param_grid.

In [19]:
ohe = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore').set_output(transform='pandas')
trf = make_column_transformer(
    (ohe, make_column_selector(dtype_include=object)), remainder='passthrough', verbose_feature_names_out=False
).set_output(transform='pandas')

In [22]:
rf = RandomForestClassifier(random_state=25)

pipe = Pipeline([
    ('trf', trf),
    ('MODEL', rf)
])


This should get error see the getparams for detail

In [ ]:
params = {
    'max_features':[2, 3, 4, 5, 6, 7, 8]
}
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=25)
gcv = GridSearchCV(pipe, param_grid=params, cv=kfold, scoring='neg_log_loss')
gcv.fit(X, y)


ValueError: Invalid parameter 'max_features' for estimator Pipeline(steps=[('trf',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000028DA8F358B0>)],
                                   verbose_feature_names_out=False)),
                ('MODEL', RandomForestClassifier(random_state=25))]). Valid parameters are: ['memory', 'steps', 'transform_input', 'verbose'].

#### We get error here because we are giving RandomForest object parameters to Pipeline.

Checking parameters of pipe

In [24]:
pipe.get_params()

{'memory': None,
 'steps': [('trf', ColumnTransformer(remainder='passthrough',
                     transformers=[('onehotencoder',
                                    OneHotEncoder(drop='first',
                                                  handle_unknown='ignore',
                                                  sparse_output=False),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x0000028DA6A873B0>)],
                     verbose_feature_names_out=False)),
  ('MODEL', RandomForestClassifier(random_state=25))],
 'transform_input': None,
 'verbose': False,
 'trf': ColumnTransformer(remainder='passthrough',
                   transformers=[('onehotencoder',
                                  OneHotEncoder(drop='first',
                                                handle_unknown='ignore',
                                                sparse_output=False),
                                  <sklearn.compose._column_transfor

In [26]:
params = {
    'MODEL__max_features':[2, 3, 4, 5, 6, 7, 8]
}
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=25)
gcv = GridSearchCV(pipe, param_grid=params, cv=kfold, scoring='neg_log_loss')
gcv.fit(X, y)

,estimator,Pipeline(step...m_state=25))])
,param_grid,"{'MODEL__max_features': [2, 3, ...]}"
,scoring,'neg_log_loss'
,n_jobs,None
,refit,True
,cv,StratifiedKFo... shuffle=True)
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('onehotencoder', ...)]"


#### So our model was in pipeline with the name 'MODEL'. So we have to prefix that name with double underscores to the parameter name or keys of the param_grid.

In [ ]:
gcv.best_score_

np.float64(-0.07789519194806123)

In [ ]:
gcv.best_params_

{'model__max_features': 3}

Saving best model

In [30]:
best_model = gcv.best_estimator_

## PICKEL 

In [27]:
import pickle 

In [32]:
pkfile = open('rf_pickle_hr.pkl', 'wb')
pickle.dump(best_model, pkfile)
pkfile.close()

Now importing and testing the pickled model

In [33]:
infile = open('./rf_pickle_hr.pkl', 'rb')
best_model_imported = pickle.load(infile)
infile.close()

In [36]:
test = pd.read_csv('../Datasets/cases/Human_Resources_Analytics/tst_hr.csv')
df = pd.DataFrame(best_model_imported.predict(test))
df

,0
0,1
1,1
2,1
3,1
4,0
5,0
6,0
7,0
8,0
9,0
